In [42]:
%pip install -q deap

Note: you may need to restart the kernel to use updated packages.


In [43]:
import pandas as pd
import ast
import random
from collections import Counter

recipes_df = pd.read_csv('recipes_revisited.csv')

In [44]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113784 entries, 0 to 113783
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                113784 non-null  int64  
 1   name                              113784 non-null  object 
 2   description                       113181 non-null  object 
 3   ingredients_raw                   113784 non-null  object 
 4   steps                             113784 non-null  object 
 5   servings                          113784 non-null  float64
 6   serving_size                      113784 non-null  object 
 7   tags                              113784 non-null  object 
 8   ingredients                       113784 non-null  object 
 9   amounts                           113784 non-null  object 
 10  amount_gram                       113784 non-null  object 
 11  serving_size_numeric              113784 non-null  f

In [45]:
# Safely parse lists from stringified columns
def safe_parse_list(x):
    try:
        return ast.literal_eval(x)
    except Exception:
        return []
recipes_df['ingredients'] = recipes_df['ingredients'].apply(safe_parse_list)

In [46]:
# Clean dataset
required_columns = [
    'recipe_proteins_per_serving',
    'recipe_carbohydrates_per_serving',
    'recipe_fat_per_serving',
    'recipe_energy_kcal_per_serving'
]
recipes_clean = recipes_df.dropna(subset=required_columns)

In [47]:
target_macros = {"protein": 150, "carbs": 200, "fat": 70, "calories": 2500}
meals_per_day = 2
planning_days = 7
total_meals = meals_per_day * planning_days
excluded_ingredients = ["peanut", "shrimp"]
preferred_ingredients = ["tomato", "beef"]

In [48]:
recipes_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108567 entries, 0 to 113783
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                108567 non-null  int64  
 1   name                              108567 non-null  object 
 2   description                       107990 non-null  object 
 3   ingredients_raw                   108567 non-null  object 
 4   steps                             108567 non-null  object 
 5   servings                          108567 non-null  float64
 6   serving_size                      108567 non-null  object 
 7   tags                              108567 non-null  object 
 8   ingredients                       108567 non-null  object 
 9   amounts                           108567 non-null  object 
 10  amount_gram                       108567 non-null  object 
 11  serving_size_numeric              108567 non-null  float6

In [49]:
from deap import base, creator, tools, algorithms

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

c:\Users\turgu\anaconda3\envs\llms\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\turgu\anaconda3\envs\llms\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [50]:
def compute_fitness(individual, recipes_df, target_macros, excluded_ingredients, preferred_ingredients):
    total_macros = {"protein": 0, "carbs": 0, "fat": 0, "calories": 0}
    used_ingredients = []
    recipe_names = []
    macro_penalty = 0

    for idx in individual:
        row = recipes_df.iloc[idx]
        protein = row['recipe_proteins_per_serving']
        carbs = row['recipe_carbohydrates_per_serving']
        fat = row['recipe_fat_per_serving']
        calories = row['recipe_energy_kcal_per_serving']

        # Penalize low-nutrition meals
        if calories < 200 or (protein + fat + carbs) < 20:
            macro_penalty += 2

        total_macros["protein"] += protein
        total_macros["carbs"] += carbs
        total_macros["fat"] += fat
        total_macros["calories"] += calories
        used_ingredients.extend(row['ingredients'])
        recipe_names.append(row['name'])

    macro_error = sum(
        abs(total_macros[k] - target_macros[k]) / target_macros[k]
        for k in target_macros
    ) / len(target_macros)

    repetition_counts = Counter(recipe_names)
    repeat_penalty = sum(count - 1 for count in repetition_counts.values() if count > 1)

    lower_ingredients = [ing.lower() for ing in used_ingredients]
    exclude_penalty = sum(
        5 for excl in excluded_ingredients if any(excl.lower() in ing for ing in lower_ingredients)
    )

    preferred_bonus = sum(
        1 for pref in preferred_ingredients if any(pref.lower() in ing for ing in lower_ingredients)
    )
    macro_error /= len(target_macros)
    repeat_penalty /= len(individual)
    macro_penalty /= len(individual)
    exclude_penalty /= len(individual)
    preferred_bonus /= len(individual)

    fitness = macro_error + repeat_penalty + macro_penalty - 0.5 * preferred_bonus + 2 * exclude_penalty
    return fitness



In [51]:
toolbox = base.Toolbox()
toolbox.register("attr_recipe", random.randint, 0, len(recipes_clean) - 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_recipe, total_meals)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", lambda ind: (compute_fitness(ind, recipes_clean, target_macros, excluded_ingredients, preferred_ingredients),))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(recipes_clean) - 1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [52]:
POP_SIZE = 50
N_GEN = 30

population = toolbox.population(n=POP_SIZE)

In [53]:
for gen in range(N_GEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.3)
    for ind in offspring:
        ind.fitness.values = toolbox.evaluate(ind)
    population = toolbox.select(offspring, k=len(population))

best_ind = tools.selBest(population, 1)[0]


In [54]:
def decode(individual, recipes_clean, meals_per_day):
    plan = {}
    for i in range(len(individual) // meals_per_day):
        meals = {}
        for j in range(meals_per_day):
            recipe = recipes_clean.iloc[individual[i * meals_per_day + j]]["name"]
            meals[f"meal_{j+1}"] = recipe
        plan[f"Day_{i+1}"] = meals
    return plan

best_plan = decode(best_ind, recipes_clean, meals_per_day)
fitness_score = best_ind.fitness.values[0]

In [55]:
import json
print("Best Plan (Fitness Score: {:.4f}):".format(fitness_score))
print(json.dumps(best_plan, indent=2))

Best Plan (Fitness Score: 0.1094):
{
  "Day_1": {
    "meal_1": "Squid and Clam Appetizer",
    "meal_2": "Wade's Apple Dumplings"
  },
  "Day_2": {
    "meal_1": "Brownie-Cherry-Ice Cream Dessert",
    "meal_2": "Coconut Gelato"
  },
  "Day_3": {
    "meal_1": "Dill Pickle-Layered Meatloaf",
    "meal_2": "Stuffed Sweet Potatoes With Mango-Black Bean Salsa"
  },
  "Day_4": {
    "meal_1": "Best Ever Sloppy Joes",
    "meal_2": "Old-Fashioned Meatloaf II"
  },
  "Day_5": {
    "meal_1": "Buca Di Beppo Italian Wedding Soup",
    "meal_2": "Greek Taboule Salad"
  },
  "Day_6": {
    "meal_1": "Strawberry Cheesecake Muffins",
    "meal_2": "Manicotti"
  },
  "Day_7": {
    "meal_1": "Jelly Roll",
    "meal_2": "Charishma's Easy Potatoes For Beginner Cooks"
  }
}


In [56]:
recipes_clean[recipes_clean['name'].isin(['Lemon Cookie Bars', 'Broiled Mahi-Mahi With Parsleyed Tomatoes and Feta'])]

,id,name,description,ingredients_raw,steps,servings,serving_size,tags,ingredients,amounts,...,recipe_proteins_per100g,recipe_fat_per100g,recipe_energy_kcal_per100g,recipe_energy_per_serving,recipe_carbohydrates_per_serving,recipe_proteins_per_serving,recipe_fat_per_serving,recipe_energy_kcal_per_serving,steps_parsed,steps_validated
40588,405308,Lemon Cookie Bars,A homemade recipe with,"['3/4 cup butter', '2 cups all-purpos...","[""Preheat oven to 350 degrees."", ""Make crust: ...",9.0,1 (85 g),"['60-minutes-or-less', 'time-to-make', 'course...","[butter, all-purpose flour, white sugar, eggs,...","[{'unit': 'cup', 'amount': 0.75}, {'unit': 'cu...",...,4.500650,14.673782,388.036590,1380.013328,52.158487,3.825553,12.472714,329.831101,"['Preheat oven to 350 degrees.', 'Make crust: ...",True
107764,140432,Broiled Mahi-Mahi With Parsleyed Tomatoes and ...,"Mahi-mahi is a very lean, moderately flavored,...","['2 onions, sliced ', '2 tablespoons ...","[""Heat olive oil over medium heat in saute pan...",6.0,1 (297 g),"['60-minutes-or-less', 'time-to-make', 'course...","[onions, olive oil, tomatoes, fresh parsley, w...","[{'unit': '', 'amount': 2.0}, {'unit': 'tables...",...,15.571274,4.392795,117.488688,1459.970835,7.817811,46.246684,13.046600,348.941404,['Heat olive oil over medium heat in saute pan...,True
